In [1]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import Field, BaseModel
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name='Search Agent',
    instructions=INSTRUCTIONS,
    model='gpt-4o-mini',
    tools=[WebSearchTool(search_context_size='low')],
    model_settings=ModelSettings(tool_choice="required")
)
message = 'Latest AI Agent frameworks in 2025'

with trace('Search agent'):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique features:

- **OpenAI Agents SDK**: Released in March 2025, this lightweight Python framework focuses on creating multi-agent workflows with comprehensive tracing and guardrails. It is compatible with over 100 different large language models (LLMs). ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: Announced in April 2025, Google's ADK is a modular framework that integrates with the Google ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and requires minimal code for efficient development. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Agent Lightning**: Introduced in August 2025, Agent Lightning is a flexible and extensible framework that enables reinforcement learning-based training of LLMs for any AI agent. It achieves complete decoupling between agent execution and training, allowing seamless integration with existing agents developed through various methods. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Also from August 2025, PCF leverages LLMs and mathematical frameworks to guide the design of adaptive AI agents for complex, dynamic environments. It enables real-time parameter reconfiguration through mathematically grounded combinatorial spaces, allowing agents to adapt their core behavioral traits dynamically. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: Introduced in July 2025, GoalfyMax is a protocol-driven framework for end-to-end multi-agent collaboration. It features a standardized Agent-to-Agent communication layer and an Experience Pack architecture, supporting robust, real-time strategy adaptation. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: Launched in August 2025, Cognitive Kernel-Pro is a fully open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It systematically investigates the curation of high-quality training data for Agent Foundation Models and explores novel strategies for agent test-time reflection and voting to enhance agent robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and methodologies for building intelligent, autonomous systems. 

In [6]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

class WebSearchItem(BaseModel):
    reason: str = Field(description='Your reasoning for why this search is important to the query.')

    query: str = Field(description='The search term to use for the web search.')

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description='A list of web searches to perform to best answer the query.')

planner_agent = Agent(
    name='Planner Agent',
    instructions=INSTRUCTIONS,
    model='gpt-4o-mini',
    output_type=WebSearchPlan,
)

message = "Latest AI Agent frameworks in 2025"

with trace('Planner Agent'):
    results = await Runner.run(planner_agent, message)

print(results.final_output)

searches=[WebSearchItem(reason='To find recent developments and the latest frameworks in AI Agents for the year 2025, including new technologies and libraries.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To identify popular AI agent frameworks and their features introduced or updated in 2025, along with user feedback or case studies.', query='top AI frameworks 2025 features'), WebSearchItem(reason='To explore emerging trends and predictions regarding AI agent technology in 2025, including insights from industry experts.', query='AI agent technology trends 2025')]


In [7]:
type(results.final_output)

__main__.WebSearchPlan

In [9]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("soroushav85@gmail.com")
    to_email = To("soroush.alahvaisi@studenti.unime.it")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [15]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print('Planning searches...')
    results = await Runner.run(planner_agent, f"Query: {query}")
    print(f'Will perform: {len(results.final_output.searches)} searches...')
    return results.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print('Performing Searches...')
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = asyncio.gather(*tasks)
    print('Results are ready...')
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    print(f'Searching for query:{item.query} ...')
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    print(f'finished Searching for {item.query}')
    return result.final_output

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print('Writing a report...')
    input = f'For this query: {query}, we found this results: {search_results}, summarize them.'
    result = await Runner.run(writer_agent, input)
    print('Finished Writing report...')
    return result.final_output


async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [16]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")



Starting research...
Planning searches...
Will perform: 3 searches...
Performing Searches...
Results are ready...
Writing a report...
Searching for query:latest AI agent frameworks 2025 ...
Searching for query:top AI companies frameworks 2025 ...
Searching for query:AI agent frameworks reviews 2025 ...
finished Searching for AI agent frameworks reviews 2025
finished Searching for latest AI agent frameworks 2025
finished Searching for top AI companies frameworks 2025
Finished Writing report...
Writing email...
Email sent
Hooray!
